In [3]:
import pandas as pd
import numpy as np
import pymssql
import warnings

warnings.filterwarnings('ignore')

In [4]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [ID Сайта], [Название склада], [Остаток], [Тип изделия 1]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ДК', N'ПДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [5]:
stock.rename(columns={'Код склада': 'name'}, inplace=True)

In [6]:
stock = stock[stock['Тип изделия 1'].str.contains('СЕРЬГИ')]

In [7]:
stock1 = stock

In [8]:
colambus = stock

In [11]:
df = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга12.xlsx')

In [12]:
net = df['здесь НЕ должно быть'].fillna(-1).astype(int).astype(str).unique().tolist()
da = df['Здесть ДОЛЖНО быть'].fillna(-1).astype(int).astype(str).unique().tolist()[:-1]

In [13]:
stock = stock[stock['ID Сайта'] != '']

In [14]:
stock = stock.query('name in @da')

In [15]:
stock = stock.groupby(by = ['ID Сайта'], as_index=False)['Остаток'].sum()

In [16]:
stock.rename(columns={'Остаток': 'Остаток на складах из списка'}, inplace=True)

In [17]:
stock1 = stock1.query('name in @net')

In [18]:
stock1 = stock1.groupby(by = ['ID Сайта'], as_index=False)['Остаток'].sum()

In [19]:
stock1 = stock1[stock1['ID Сайта'] != '']

In [20]:
stock = stock.merge(stock1, how='left', on='ID Сайта')

In [21]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7776 entries, 0 to 7775
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID Сайта                      7776 non-null   object 
 1   Остаток на складах из списка  7776 non-null   float64
 2   Остаток                       6083 non-null   float64
dtypes: float64(2), object(1)
memory usage: 182.4+ KB


In [22]:
stock['Остаток'] = stock['Остаток'].fillna(-1)

In [23]:
stock = stock[stock['Остаток'] < 0]

In [24]:
stock.drop(columns=['Остаток'], inplace=True)

In [25]:
colambus = colambus[colambus['name'] == '5442']

In [26]:
colambus = colambus.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()

In [27]:
stock = stock.merge(colambus, how='left', on='ID Сайта')

In [28]:
stock.rename(columns={'Остаток': 'Остаток на коламбусе'}, inplace=True)

In [29]:
stock.to_excel('Остатки по id.xlsx', index=False)